In [1]:
from Bio.PDB import PDBParser
import torch
from qres.structure_prediction import StructurePredictor

sequences = ["IWVRNSRNWWWWVTMNDWNN", "IWVRNSRNWWWWVTMNLWNN"]
sequence = sequences[0]

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
predictor = StructurePredictor(device=device)

# Predict the structure
pdb_str = predictor.predict_structure([sequence])[0]

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(pdb_str)

PARENT N/A
ATOM      1  N   ILE A   1      -0.856  -9.720 -25.351  1.00  0.67           N  
ATOM      2  CA  ILE A   1      -0.431 -10.549 -24.229  1.00  0.68           C  
ATOM      3  C   ILE A   1      -0.932  -9.943 -22.920  1.00  0.68           C  
ATOM      4  CB  ILE A   1       1.106 -10.707 -24.194  1.00  0.61           C  
ATOM      5  O   ILE A   1      -0.609  -8.798 -22.595  1.00  0.65           O  
ATOM      6  CG1 ILE A   1       1.614 -11.285 -25.519  1.00  0.54           C  
ATOM      7  CG2 ILE A   1       1.531 -11.586 -23.014  1.00  0.55           C  
ATOM      8  CD1 ILE A   1       3.109 -11.102 -25.742  1.00  0.54           C  
ATOM      9  N   TRP A   2      -2.281  -9.990 -22.452  1.00  0.71           N  
ATOM     10  CA  TRP A   2      -2.790  -9.542 -21.160  1.00  0.73           C  
ATOM     11  C   TRP A   2      -1.999 -10.170 -20.017  1.00  0.70           C  
ATOM     12  CB  TRP A   2      -4.275  -9.885 -21.019  1.00  0.66           C  
ATOM     13  O   

In [61]:
import py3Dmol
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [69]:

palette = matplotlib.colormaps["magma_r"]
print(palette.N)

256


In [77]:

view = py3Dmol.view(width=800, height=600)

# Add the structure from the PDB string
view.addModel(pdb_str, "pdb")

i = 0
bs = []
idxs = []
for line in pdb_str.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    b = float(split[-2])
    assert b >= 0 and b <= 1
    bs.append(b)

    idx = int(split[1])
    idxs.append(idx)
    i += 1

bs = np.array(bs)
bs = (bs - bs.min())
bs = bs / bs.max()

print(bs.max())
print(bs.min())

colors = []

for b, idx in zip(bs, idxs):
    # print(b)
    color = '0x' + ''.join([hex(int(x*255))[2:].zfill(2) for x in palette(b)[:3]])
    colors.append(color[2:])
    view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})

    view.zoomTo()

view.show()
png = view.png()

1.0
0.0


3Dmol.js failed to load for some reason. Please check your browser console for error messages.